## ⚙️Prerequisites
### Setup SSCMA
Clone the [repository](https://github.com/Seeed-Studio/ModelAssistant) and install the dependencies.

In [ ]:
!rm -rf ModelAssistant
%env PYTHON_EXEC=python3.8
!git clone https://review.mlplatform.org/ml/ethos-u/ethos-u-vela.git
%cd ethos-u-vela
!pip install .
%cd ..
!git clone https://github.com/Seeed-Studio/ModelAssistant.git   #clone the repo

%cd ModelAssistant
!git status
!. ./scripts/setup_colab.sh

### Download the pretrain model weights file

In [ ]:
%rm -rf pilboi-yolo/
%mkdir -p pilboi-yolo
!wget -c https://files.seeedstudio.com/sscma/model_zoo/detection/coco/swift_yolo_shuffle_coco_320_float32_sha1_a5927bd6a6c6569d27edb98da946a8e75a8d816f.pth -O pilboi-yolo/pretrain.pth

### Download the dataset

In [ ]:

%mkdir -p pilboi-yolo/dataset
!wget -c https://app.roboflow.com/ -O pilboi-yolo/dataset.zip
!unzip -q pilboi-yolo/dataset.zip -d pilboi-yolo/dataset

## 🚀Train a model with SSCMA
All the training parameters are in the `config.py` file, you can change the parameters to train your own model.

Below are explanations of some common parameters. You can also refer to the [documentation](https://sensecraftma.seeed.cc/tutorials/config) for more details.
- `data_root` - the datasets path.
- `epochs`- the train epochs. **we use 10 epochs as an example**.
- `batch_size` - the batch size.
- `height` - the image height.
- `width` - the image width.
- `load_from` - the pretrained model path.
- `num_classes` - the number of classes.

You can overwrite the parameters in the `config.py` file by using the `--cfg-options` argument.
```bash
# Example
sscma.train config.py --cfg-options data_root=./datasets/test_dataset epochs=10
```

In [ ]:
!pwd
!cd /content/ModelAssistant/
!ls pilboi-yolo/dataset/valid
!sscma.train configs/swift_yolo/swift_yolo_shuff_1xb16_300e_coco.py \
--cfg-options  \
    work_dir=pilboi-yolo \
    num_classes=8 \
    epochs=100  \
    height=256 \
    width=256 \
    data_root=pilboi-yolo/dataset/ \
    load_from=pilboi-yolo/pretrain.pth

## 📦Export the model
After training, you can export the model to the format for deployment. SSCMA supports exporting to ONNX, and TensorFlow Lite at present.
You can also refer to the [documentation](https://sensecraftma.seeed.cc/tutorials/export/overview) for more details.

```bash
python3 tools/export.py \
    "<CONFIG_FILE_PATH>" \
    "<CHECKPOINT_FILE_PATH>"
```

In [ ]:
import os
with open('pilboi-yolo/last_checkpoint', 'r') as f:
	os.environ['CHECKPOINT_FILE_PATH'] = f.read()

In [ ]:
!sscma.export configs/swift_yolo/swift_yolo_shuff_1xb16_300e_coco.py $CHECKPOINT_FILE_PATH --cfg-options  \
    work_dir=pilboi-yolo \
    num_classes=8 \
    epochs=100  \
    height=256 \
    width=256 \
    data_root=pilboi-yolo/dataset/ \
    load_from=pilboi-yolo/pretrain.pth

### 📝Evaluate the model
After exporting the model, you can evaluate the model on the test dataset.
You can also refer to the [documentation](https://sensecraftma.seeed.cc/tutorials/export/overview) for more details.


```bash
python3 tools/inference.py \
    "<CONFIG_FILE_PATH>" \
    "<CHECKPOINT_FILE_PATH>"
```

### Evaluate the PyTorch model

In [ ]:
!sscma.inference configs/swift_yolo/swift_yolo_shuff_1xb16_300e_coco.py ${CHECKPOINT_FILE_PATH%.*}.pth \
--cfg-options  \
    work_dir=pilboi-yolo \
    num_classes=8 \
    epochs=70  \
    height=256 \
    width=256 \
    data_root=pilboi-yolo/dataset/ \
    load_from=pilboi-yolo/pretrain.pth

### Evaluate the ONNX model

In [ ]:
!sscma.inference configs/yolov5/swift_yolo_shuff_1xb16_300e_coco.py ${CHECKPOINT_FILE_PATH%.*}_float32.onnx \
--cfg-options  \
    work_dir=COCO_Detection_Swift-YOLO_320 \
    num_classes=1 \
    epochs=10  \
    height=320 \
    width=320 \
    data_root=COCO_Detection_Swift-YOLO_320/dataset/ \
    load_from=COCO_Detection_Swift-YOLO_320/pretrain.pth

### Evaluate the TFLite FLOAT32 model

In [ ]:
!sscma.inference configs/yolov5/swift_yolo_shuff_1xb16_300e_coco.py ${CHECKPOINT_FILE_PATH%.*}_float32.tflite \
--cfg-options  \
    work_dir=COCO_Detection_Swift-YOLO_320 \
    num_classes=1 \
    epochs=10  \
    height=320 \
    width=320 \
    data_root=COCO_Detection_Swift-YOLO_320/dataset/ \
    load_from=COCO_Detection_Swift-YOLO_320/pretrain.pth

### Evaluate the TFLite INT8 model

In [ ]:
!sscma.inference configs/swift_yolo/swift_yolo_shuff_1xb16_300e_coco.py ${CHECKPOINT_FILE_PATH%.*}_int8.tflite \
--cfg-options  \
    work_dir=pilboi-yolo \
    num_classes=8 \
    epochs=100  \
    height=256 \
    width=256 \
    data_root=pilboi-yolo/dataset/ \
    load_from=pilboi-yolo/pretrain.pth

## 🤖 Deploy the model
After model training, evaluation and export, you can deploy the model to your device. You can refer to [Documentation](https://sensecraftma.seeed.cc/deploy/overview) for more details.

In [ ]:
from google.colab import files
files.download(f'pilboi-yolo/epoch_100_int8_vela.tflite')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>